In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline
model_name = 'qanastek/51-languages-classifier'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)


In [1]:
# classifier = TextClassificationPipeline(model=model, tokenizer=tokenizer)
# res = classifier("hello how are you ?")
# print(res)

In [4]:
# from transformers import AutoTokenizer

# # tokenizer = AutoTokenizer.from_pretrained("qanastek/51-languages-classifier")

# code_sample = "i am maninder')"

# tokens = tokenizer(code_sample, return_tensors="pt")
# print(tokens)



In [89]:
import datasets
data = datasets.load_from_disk('preprocessed_programming_language_data')
data = data.shuffle(seed= 42)
# data = data.train_test_split(0.1)

In [90]:
data

Dataset({
    features: ['code', 'language_name'],
    num_rows: 100816
})

In [91]:
# data = datasets.load_dataset('pandas','preprocessed_programming_language_data.csv')
data = data.rename_column('code','text')
data = data.rename_column('language_name','label')

In [92]:
tokenized_datasets = data.map(lambda examples: tokenizer(examples['text'], truncation=True, padding=True), batched=True)

In [93]:
# tokenized_datasets['input_ids']

In [94]:
# labels = set(data["label"])
# # print(labels)
# label2id, id2label = dict(), dict()
# for i, label in enumerate(labels):
#     label2id[label] = str(i)
#     id2label[str(i)] = label

In [95]:
# 'H' in labels

In [96]:
# set(data['language_name'])
data

Dataset({
    features: ['text', 'label'],
    num_rows: 100816
})

In [97]:
# data[0]

In [98]:
# Convert string labels to numerical format
# label_mapping = {label: i for i, label in enumerate(set(data['language_name']))}
# tokenized_datasets = tokenized_datasets.map(lambda examples: {'label': [label_mapping[label] for label in examples['language_name']]})
# label_mapping
# Convert string labels to numerical format
label_mapping = {label: i for i, label in enumerate(set(data['label']))}
# print("Label mapping:", label_mapping)

def map_labels(examples):
    if 'label' in examples:
        # try:
            # print(examples)
            # return {'label': [label_mapping[label] for label in examples['label']]}
        return {'label':label_mapping[examples['label']]}
        # except:
        #     print(examples['label'])
    else:
        return {'label': []}  # or any other appropriate handling

tokenized_datasets = tokenized_datasets.map(map_labels)


Map:   0%|          | 0/100816 [00:00<?, ? examples/s]

In [104]:
tokenized_datasets[0]['label'],label_mapping['Haskell']

(276, 276)

In [105]:
id2label = {y: x for x, y in label_mapping.items()}

In [106]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
import evaluate

accuracy = evaluate.load("accuracy")
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [108]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=len(set(data['label'])), id2label=id2label, label2id=label_mapping,ignore_mismatched_sizes=True
)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at qanastek/51-languages-classifier and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([51, 768]) in the checkpoint and torch.Size([1006, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([51]) in the checkpoint and torch.Size([1006]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [110]:
from sklearn.model_selection import train_test_split
tokenized_datasets_splited = tokenized_datasets.train_test_split(test_size=0.2, seed=42)

In [111]:
tokenized_datasets_splited

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 80652
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 20164
    })
})

In [112]:
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets_splited["train"],
    eval_dataset=tokenized_datasets_splited["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 